In [232]:

import subprocess  # subprocess 모듈을 가져옴
import sys

# torch 모듈을 임시로 설치하는 코드
try:
    import torch
except ImportError:
    print("Torch is not installed. Installing torch...")
    subprocess.check_call([sys.executable, "-m", "pip", "install", "torch"])
    import torch

# 이후 torch를 사용한 코드 작성
print(torch.__version__)



def get_random_problems(batch_size, problem_size): # size를 입력으로 받아 무작위로 생성된 경로 문제를 반환
    #batch_size = 동시에 생성할 문제의 개수
    #problem_size = 문제 내의 고객 노드의 수(방문해야 할 위치의 수)

    depot_xy = torch.full((batch_size, 1, 2), 0.5) # 나중에 depot 고정해야하니까 잊지 말기!!!
    # shape: (batch, 1, 2) # (문제의 개수, depot의 개수, 2차원)

    node_xy = torch.rand(size=(batch_size, problem_size, 2))
    # shape: (batch, problem, 2)  # (문제의 개수, node의 개수, 2차원)

    if problem_size == 10:
        demand_scaler = 15
    elif problem_size == 20:
        demand_scaler = 30
    elif problem_size == 50:
        demand_scaler = 40
    elif problem_size == 100:
        demand_scaler = 50 ## 현재 problem size가 20 50 100으로 고정되어 있는데 이건 나중에 바꿔도 됨
    else:
        raise NotImplementedError

    node_demand = torch.rand(size=(batch_size, problem_size)) * 0.3

    return depot_xy, node_xy, node_demand



def augment_xy_data_by_8_fold(xy_data):
    # xy_data.shape: (batch, N, 2) # 입력된 좌표 데이터를 8배로 증강하여 반환
    # N은 각 문제에 포함된 노드의 수를 의미

    x = xy_data[:, :, [0]] #모든 (x,y) 좌표중 x값만 뽑아냄
    y = xy_data[:, :, [1]] #모든 (x,y) 좌표중 y값만 뽑아냄
    # x,y shape: (batch, N, 1)

    dat1 = torch.cat((x, y), dim=2) #원본 데이터
    dat2 = torch.cat((1 - x, y), dim=2) # x축 기준 좌표 반전
    dat3 = torch.cat((x, 1 - y), dim=2) # y축 기준 좌표 반전
    dat4 = torch.cat((1 - x, 1 - y), dim=2) # 원점 기준 좌표 반전
    dat5 = torch.cat((y, x), dim=2) # y=x 직선에 대한 반전
    dat6 = torch.cat((1 - y, x), dim=2) # y=-x 직선에 대한 반전
    dat7 = torch.cat((y, 1 - x), dim=2) # x 축 반전 후 y= x에 대해 반전
    dat8 = torch.cat((1 - y, 1 - x), dim=2) # 원점 기준 좌표 반전 후 y = x 직선에 대해 반전

    aug_xy_data = torch.cat((dat1, dat2, dat3, dat4, dat5, dat6, dat7, dat8), dim=0)
    # shape: (8*batch, N, 2)
    # 한 데이터로 8개의 데이터를 만듦으로 총 데이터의 shape 은 (8*B, N, 2)
 
    return aug_xy_data


2.3.1


In [233]:

from dataclasses import dataclass
import torch
import sys
import os
from CVRProblemDef import get_random_problems, augment_xy_data_by_8_fold




@dataclass #dataclass는 데이터 중심의 클래스 정의를 더 간결하고 명확하게 만들어주는 도구임
class Reset_State: #초기화 시의 상태 저장 클래스
    depot_xy: torch.Tensor = None
    # shape: (batch, 1, 2)
    node_xy: torch.Tensor = None
    # shape: (batch, problem, 2)
    node_demand: torch.Tensor = None
    # shape: (batch, problem)


@dataclass
class Step_State: #매 단계마다의 상태 저장 클래스
    BATCH_IDX: torch.Tensor = None
    POMO_IDX: torch.Tensor = None
    # shape: (batch, pomo)
    selected_count: int = None #현재까지 남아있는 노드의 수
    load: torch.Tensor = None #현재 남아있는 적재량 (추후에 현재 남아있는 배터리량도 넣어줘야함)
    # shape: (batch, pomo)
    soc: torch.Tensor = None #현재 남아있는 배터리량
    # shape: (batch, pomo)
    current_node: torch.Tensor = None #현재 위치한 노드
    # shape: (batch, pomo)
    ninf_mask: torch.Tensor = None #방문 불가능한 노드나 이미 방문한 노드 표시
    # shape: (batch, pomo, problem+1)
    finished: torch.Tensor = None #각 그룹의 작업이 완료되었는지 판단
    # shape: (batch, pomo)


class CVRPEnv: #환경을 설정하고 데이터를 관리하며 상태를 추적하고 업데이트하는 역할을 하는 클래스
    def __init__(self, **env_params):

        # Const @INIT
        ####################################
        self.env_params = env_params #환경 매개변수 저장
        self.problem_size = env_params['problem_size'] #문제의 크기
        self.pomo_size = env_params['pomo_size'] #pomo의 크기
        self.initial_battery = 100

        self.FLAG__use_saved_problems = False #저장된 문제 데이터를 사용할지 여부(기본값은 false)
        self.saved_depot_xy = None # 얻어진 depot의 좌표를 저장할 변수
        self.saved_node_xy = None # 얻어진 고객 노드들의 좌표를 저장할 변수
        self.saved_node_demand = None #얻어진 고객 노드들의 수요를 저장할 변수
        self.saved_index = None # 얻어진 문제 데이터의 인덱스를 추적하는 변수

        # Const @Load_Problem
        ####################################
        self.batch_size = None # 배치 크기를 저장하는 변수
        self.BATCH_IDX = None # 배치 인덱스를 저장하는 텐서
        self.POMO_IDX = None # POMO 인덱스를 저장하는 텐서(에이전트의 인덱스)
        # IDX.shape: (batch, pomo) 
        self.depot_node_xy = None #depot과 node의 좌표를 포함하는 텐서
        # shape: (batch, problem+1, 2)
        self.depot_node_demand = None #depot과 node의 수요를 포함하는 텐서
        # shape: (batch, problem+1)

        # Dynamic-1
        ####################################
        self.selected_count = None #현재까지 선택된 노드의 수를 저장하는 변수
        self.current_node = None #현재 선택된 노드를 저장하는 변수
        # shape: (batch, pomo)
        self.selected_node_list = None # 현재까지 선택된 노드들이 순서대로 저장되는 노드 리스트 변수
        # shape: (batch, pomo, 0~)

        # Dynamic-2
        ####################################
        self.at_the_depot = None #각 에이전트가 현재 depot에 있는지 나타내는 변수
        # shape: (batch, pomo)
        self.load = None #각 에이전트가 현재 드론에 남아있는 용량을 저장하는 변수
        # shape: (batch, pomo)
        self.soc = None
        # shape: (batch, pomo)
        self.visited_ninf_flag = None #방문한 노드들에 대해 마스크를 적용하는 플래그 변수
        # shape: (batch, pomo, problem+1)
        self.ninf_mask = None #선택 불가능한 노드에 대해 마스크를 적용하는 변수
        # shape: (batch, pomo, problem+1)
        self.finished = None #각 에이전트의 여행이 완료되었는지를 나타내는 변수
        # shape: (batch, pomo)

        # states to return
        ####################################
        self.reset_state = Reset_State() #환경을 초기화한 상태를 저장하기 위한 객체
        self.step_state = Step_State() # 각 단계에서의 상태를 저장하기 위한 객체

    def use_saved_problems(self, filename, device): # 이전에 생성된 문제 데이터를 로드하여 사용할 수 있도록 설정하는 역할
        self.FLAG__use_saved_problems = True # 저장된 문제 데이터를 사용할지 여부를 나타내는 변수로 기본 값은 True

        loaded_dict = torch.load(filename, map_location=device) #저장된 데이터 불러옴
        self.saved_depot_xy = loaded_dict['depot_xy'] #저장된 데이터의 디폿 값
        self.saved_node_xy = loaded_dict['node_xy'] # 저장된 데이터의 노드값
        self.saved_node_demand = loaded_dict['node_demand'] #저장된 데이터의 수요값
        self.saved_index = 0 #로드된 데이터를 사용하기 위한 인덱스 초기화(이건 depot 고정일때 다시 한번 생각해 봐야함)

    def load_problems(self, batch_size, aug_factor=1): #문제 데이터를 로드하거나 필요에 따라 데이터를 증강하는 역할을 함
        self.batch_size = batch_size #배치사이즈를 클래스 인스턴스의 self.batch_size 변수에 저장

        if not self.FLAG__use_saved_problems: # 저장된 문제 데이터를 사용할지 여부를 결정
            depot_xy, node_xy, node_demand = get_random_problems(batch_size, self.problem_size) #false일 경우 새 문제 데이터 생성
        else: # true일 경우 저장된 문제 데이터 로드
            depot_xy = self.saved_depot_xy[self.saved_index:self.saved_index+batch_size]
            node_xy = self.saved_node_xy[self.saved_index:self.saved_index+batch_size]
            node_demand = self.saved_node_demand[self.saved_index:self.saved_index+batch_size]
            self.saved_index = self.saved_index + batch_size # 저장된 인덱스 이후의 데이터를 사용할 수 있도록 함

        if aug_factor > 1: #데이터 증강 배율이 1보다 크면 데이터 증강 수행
            if aug_factor == 8: #8배인경우 배치 크기 8배로 설정
                self.batch_size = self.batch_size * 8
                depot_xy = augment_xy_data_by_8_fold(depot_xy)
                node_xy = augment_xy_data_by_8_fold(node_xy)
                node_demand = node_demand.repeat(8, 1)
            else: # 1과 8 둘다 아니면 예외를 발생
                raise NotImplementedError

        self.depot_node_xy = torch.cat((depot_xy, node_xy), dim=1) #depot과 node의 좌표를 결합하여 저장
        # shape: (batch, problem+1, 2)
        depot_demand = torch.zeros(size=(self.batch_size, 1)) #차량 기지에 대해서는 수요를 0으로 설정
        # shape: (batch, 1)
        self.depot_node_demand = torch.cat((depot_demand, node_demand), dim=1) #차량 기지의 수요와 고객 노드들의 수요를 결합하여 총 수요를 저장
        # shape: (batch, problem+1)


        self.BATCH_IDX = torch.arange(self.batch_size)[:, None].expand(self.batch_size, self.pomo_size) #각 배치의 인덱스를 반복하여 pomo 크기만큼 확장한 인덱스 텐서 생성
        self.POMO_IDX = torch.arange(self.pomo_size)[None, :].expand(self.batch_size, self.pomo_size) #pomo의 인덱스를 배치 크기만큼 확장한 인덱스 텐서를 생성

        self.reset_state.depot_xy = depot_xy #depot.xy를 reset_state 객체에 저장
        self.reset_state.node_xy = node_xy #node.xy를 reset_state 객체에 저장
        self.reset_state.node_demand = node_demand #node_demand를 reset_state 객체에 저장

        self.step_state.BATCH_IDX = self.BATCH_IDX #이전에 생성한 batch_idx를 step_state개체의 batch_idx 속성에 저장
        self.step_state.POMO_IDX = self.POMO_IDX #이전에 생성한 pomo_idx를 step_state개체의 pomo_idx 속성에 저장
        
        # batch : 여러 문제 인스턴스를 동시에 처리하여 학습 속도를 높임
        # POMO :각 문제 인스턴스에 대해 다양한 경로를 생성하여 더 나은 솔루션을 찾도록 도와줌

    def reset(self): #환경 상태를 초기화하여 새로운 에피소드를 시작할 준비를 함
        self.selected_count = 0 # 선택된 노드의 개수를 0으로 초기화
        self.current_node = None # 현재 선택된 노드를 none으로 설정하여 아직 어떤 노드도 설정되지 않았음을 나타냄
        # shape: (batch, pomo)
        self.selected_node_list = torch.zeros((self.batch_size, self.pomo_size, 0), dtype=torch.long) #선택된 노드들의 리스트를 저장하는 역할을 하며 초기화
        # shape: (batch, pomo, 0~)

        self.at_the_depot = torch.ones(size=(self.batch_size, self.pomo_size), dtype=torch.bool) #모든 pomo의 위치가 depot에 있음을 나타냄
        # shape: (batch, pomo)
        self.load = torch.ones(size=(self.batch_size, self.pomo_size)) # 모든 pomo가 가지고 있는 용량을 나타냄
        # shape: (batch, pomo)
        self.soc = torch.ones(size=(self.batch_size, self.pomo_size)) * (self.initial_battery - 5)  # 초기 배터리 잔량을 100으로 설정 (예시)
        # shape: (batch, pomo)
        self.visited_ninf_flag = torch.zeros(size=(self.batch_size, self.pomo_size, self.problem_size+1)) #각 노드가 방문되었는지 여부를 나타내며 모든 요소가 0으로 초기화
        # shape: (batch, pomo, problem+1)
        self.ninf_mask = torch.zeros(size=(self.batch_size, self.pomo_size, self.problem_size+1)) #선택할 수 없는 노드에 대해 마스킴하며 모든 요소가 0으로 초기화
        # shape: (batch, pomo, problem+1)
        self.finished = torch.zeros(size=(self.batch_size, self.pomo_size), dtype=torch.bool) # 각 POMO가 작업을 완료했는지 여부를 나타내며 모든 요소가 false로 초기화
        # shape: (batch, pomo)

        reward = None #초기에는 보상이 없으므로 none
        done = False #에피소드가 완료되지 않았으므로 false
        return self.reset_state, reward, done # 초기화된 상태의 보상과 완료 여부를 반환

    def pre_step(self): #reset_state를 step_state에 저장하고 다음 단계를 수행하기 전 상태를 업데이트하도록 준비하는 역할
        self.step_state.selected_count = self.selected_count # 현재까지 선택된 노드의 개수를 나타냄
        self.step_state.load = self.load # 각 POMO가 현재 가지고 있는 용량을 나타냄
        self.step_state.soc = self.soc # 각 드론의 배터리 잔량 업데이트
        self.step_state.current_node = self.current_node # 현재 선택된 노드를 나타냄
        self.step_state.ninf_mask = self.ninf_mask # 선택할 수 없는 노드를 마스킹하기 위한 텐서
        self.step_state.finished = self.finished # 각 POMO가 작업을 완료했는지 여부를 나타냄

        reward = None #보상을 none으로 초기화
        done = False #에피소드가 완료되지 않았음을 표현
        return self.step_state, reward, done #현재 상태 보상 완료상태를 반환

    def step(self, selected): 
        # 선택된 노드의 거리를 계산하고 배터리 소모량을 업데이트
        segment_distances = self._get_segment_distances()
        
        # 각 구간의 거리에 따라 배터리 소모량 계산
        for i in range(self.selected_node_list.size(2) - 1):
            current_distances = segment_distances[:, :, i]
            self.soc = self.soc - self.calculate_soc(self.load, current_distances)  # 배터리 소모 적용

        # 디버깅: 배터리 소모가 적용된 후의 SoC 출력
        print(f"Updated SoC after applying segment distances: {self.soc}")

        # Dynamic-1: 노드 선택과 업데이트
        self.selected_count += 1  
        self.current_node = selected  
        self.selected_node_list = torch.cat((self.selected_node_list, self.current_node[:, :, None]), dim=2)

        # Dynamic-2: 적재량과 배터리 업데이트
        self.at_the_depot = (selected == 0)
        demand_list = self.depot_node_demand[:, None, :].expand(self.batch_size, self.pomo_size, -1)
        gathering_index = selected[:, :, None]
        selected_demand = demand_list.gather(dim=2, index=gathering_index).squeeze(dim=2)

        # Load 업데이트
        self.load = self.load - selected_demand
        self.load[self.at_the_depot] = 1  # depot에 있는 드론은 적재량 리필

        # depot 복귀 판단
        distance_to_depot = self.calculate_distance_to_depot(selected)
        battery_consumption_to_depot = self.calculate_soc(self.load, distance_to_depot)
        remaining_battery_after_return = self.soc - battery_consumption_to_depot

        # 배터리가 부족한 드론은 depot으로 복귀
        selected = selected.clone()
        selected[(remaining_battery_after_return < 15) & (~self.at_the_depot)] = 0
        self.soc[self.at_the_depot] = self.initial_battery - 5  # depot에서 배터리 리필

        # 디버깅: 배터리 리필 후 SoC 출력
        print(f"Updated SoC after potential depot return: {self.soc}")

        # 상태 업데이트
        self.step_state.selected_count = self.selected_count
        self.step_state.load = self.load
        self.step_state.soc = self.soc
        self.step_state.current_node = self.current_node
        self.step_state.ninf_mask = self.ninf_mask
        self.step_state.finished = self.finished

        # 완료 여부 반환
        done = self.finished.all()
        reward = -self._get_travel_distance() if done else None

        return self.step_state, reward, done




    
    def calculate_soc(self, payload, distances):
        # 배터리 소모율 계산
        soc_consumption = torch.zeros_like(payload)

        # 각 payload 구간에 따라 BCR 적용
        soc_consumption = soc_consumption + torch.logical_and(payload >= 0, payload < 0.2) * (-2.29705 * 0.1 + 3.87886) * distances
        soc_consumption = soc_consumption + torch.logical_and(payload >= 0.2, payload < 0.4) * (-2.29705 * 0.3 + 3.87886) * distances
        soc_consumption = soc_consumption + torch.logical_and(payload >= 0.4, payload < 0.6) * (-2.29705 * 0.5 + 3.87886) * distances
        soc_consumption = soc_consumption + torch.logical_and(payload >= 0.6, payload < 0.8) * (-2.29705 * 0.7 + 3.87886) * distances
        soc_consumption = soc_consumption + torch.logical_and(payload >= 0.8, payload <= 1.0) * (-2.29705 * 0.9 + 3.87886) * distances

        return soc_consumption
    
    #def calculate_travel_time(self, current_node):
        # 노드 간의 거리와 속도에 따라 시간을 계산합니다.
    #   distances = self._get_travel_distance()
    #  time_per_node = distances / 22.0  # 속도 22km/h로 고정
    #    return time_per_node
    
    def calculate_distance_to_depot(self, selected):
        # 선택된 노드에서 디팟까지의 거리를 계산하는 함수
        depot_xy = self.depot_node_xy[:, 0, :].unsqueeze(1).expand(self.batch_size, selected.size(1), -1)  # (batch, pomo, 2)로 확장
        node_xy = self.depot_node_xy[self.BATCH_IDX, selected]  # 선택된 노드의 좌표

        distance_to_depot = torch.sqrt(((node_xy - depot_xy) ** 2).sum(dim=-1))  # 유클리드 거리 계산
        return distance_to_depot
    
    def _get_segment_distances(self):
            gathering_index = self.selected_node_list[:, :, :, None].expand(-1, -1, -1, 2)
            # shape: (batch, pomo, selected_list_length, 2)
    
            all_xy = self.depot_node_xy[:, None, :, :].expand(-1, self.pomo_size, -1, -1)
            # shape: (batch, pomo, problem+1, 2)

            ordered_seq = all_xy.gather(dim=2, index=gathering_index)
            # 선택된 노드들의 좌표를 순서대로 나열
            # shape: (batch, pomo, selected_list_length, 2)

            rolled_seq = ordered_seq.roll(dims=2, shifts=-1)
            # ordered_seq를 한 칸씩 이동하여 각 구간의 거리를 계산하기 위한 준비
            # shape: (batch, pomo, selected_list_length, 2)

            segment_lengths = ((ordered_seq - rolled_seq) ** 2).sum(3).sqrt()
            # 각 구간의 유클리드 거리 계산
            # shape: (batch, pomo, selected_list_length)

            return segment_lengths
        
    def _get_travel_distance(self):
        gathering_index = self.selected_node_list[:, :, :, None].expand(-1, -1, -1, 2) 
        # 마지막에 2인 이유는 (x,y)의 노드 인덱스를 가져오기 위함임
        # shape: (batch, pomo, selected_list_length, 2)
        all_xy = self.depot_node_xy[:, None, :, :].expand(-1, self.pomo_size, -1, -1) 
        #각 배치와 pomo인덱스에 대해 모든 노드의 좌표를 반복하여 확장한 텐서
        # shape: (batch, pomo, problem+1, 2)

        ordered_seq = all_xy.gather(dim=2, index=gathering_index) 
        #gathering_index를 사용하여 all xy 텐서에서 각 노드의 좌표를 추출하며 이 코드는 선택된 노드들의 좌표를 순서대로 나열하겠다는 의미
        # shape: (batch, pomo, selected_list_length, 2)

        rolled_seq = ordered_seq.roll(dims=2, shifts=-1) 
        #ordered_seq을 한 칸씩 왼쪽으로 이동시킨 텐서로 마지막 노드는 처음으로 이동
        segment_lengths = ((ordered_seq-rolled_seq)**2).sum(3).sqrt() 
        #ordered_seq과 rolled_seq사이의 차이를 구하고 이를 제곱한 후 마지막 차원에서 제곱하여 마지막 차원에서 합산하여 각 구간의 거리의 제곱을 계산
        # shape: (batch, pomo, selected_list_length)

        travel_distances = segment_lengths.sum(2) #각 구간의 거리를 합산하여 총 이동 거리를 계산
        # shape: (batch, pomo)
        
        return travel_distances



In [234]:
import plotly.graph_objs as go
import torch
from CVRPModel import CVRPModel
import numpy as np

# 환경 및 모델 설정
env_params = {
    'problem_size': 10,  # 노드 수
    'pomo_size': 5       # POMO 수 (여러 대의 드론이 각기 다른 경로를 탐색)
}

model_params = {
    'embedding_dim': 128,
    'sqrt_embedding_dim': 128**(1/2),
    'encoder_layer_num': 6,
    'qkv_dim': 16,
    'head_num': 8,
    'logit_clipping': 10,
    'ff_hidden_dim': 512,
    'eval_type': 'argmax'
}

# CVRP 환경 및 모델 생성
env = CVRPEnv(**env_params)
model = CVRPModel(**model_params)

# 문제 로드 및 초기화
env.load_problems(batch_size=1)
reset_state, reward, done = env.reset()

# 모델에 상태를 전달하여 초기화
model.pre_forward(reset_state)
print("\n--- Debugging reset() ---")
print("Depot XY:", reset_state.depot_xy)
print("Node XY:", reset_state.node_xy)
print("Node Demand:", reset_state.node_demand)
print("Reward:", reward)
print("Done:", done)
print("--------------------------\n")

# 첫 번째 단계에서 선택 (출발지, depot)
step_state, reward, done = env.pre_step()
print("\n--- Debugging pre_step() ---")
print("Selected Count:", step_state.selected_count)
print("Load:", step_state.load)
print("Battery (SoC):", step_state.soc)
print("Current Node:", step_state.current_node)
print("Ninf Mask:", step_state.ninf_mask)
print("Finished:", step_state.finished)
print("Reward:", reward)
print("Done:", done)
print("-----------------------------\n")

# 경로를 저장할 리스트
drone_paths = [[] for _ in range(env_params['pomo_size'])]
depot_xy = env.reset_state.depot_xy[0, 0].cpu().numpy()  # depot 좌표가 2차원일 때 처리
node_positions = env.depot_node_xy[0].cpu().numpy()  # 모든 노드의 좌표
node_demands = env.reset_state.node_demand[0].cpu().numpy()  # 모든 노드의 demand 값

# 선택 카운터 증가
selection_counter = 0

# 계속해서 모든 노드를 방문하고 디팟으로 돌아올 때까지 선택하고 상태를 업데이트
while True:
    # 모델로부터 다음 선택을 받음
    selected, prob = model(step_state)

    # 선택된 노드를 각 드론 경로에 저장
    for pomo_idx in range(env_params['pomo_size']):
        selected_node_xy = env.depot_node_xy[0, selected[0, pomo_idx]].cpu().numpy()
        drone_paths[pomo_idx].append(selected_node_xy)
    
    # 선택한 노드에 대한 수요(demand) 가져오기
    selected_demand = env.depot_node_demand[0, selected].cpu().numpy()

    print(f"\nSelection {selection_counter}:")
    print("Selected Nodes:", selected)
    print("Probabilities:", prob)
    print("Remaining Load:", step_state.load)
    print("Remaining Battery (SoC):", step_state.soc)
    print("Demand at Selected Nodes:", selected_demand)

    # 이전 노드와 선택된 노드 간의 거리 계산
    if step_state.current_node is not None:
        previous_node_xy = env.depot_node_xy[0, step_state.current_node[:, :, None]].squeeze(2).cpu().numpy()
        selected_node_xy = env.depot_node_xy[0, selected].cpu().numpy()

        # 유클리드 거리 계산
        distance_to_previous_node = ((previous_node_xy - selected_node_xy) ** 2).sum(axis=-1) ** 0.5
        print("Distance from Previous Node:", distance_to_previous_node)

    # 선택된 노드에 대한 상태 업데이트
    step_state, reward, done = env.step(selected)

    # 선택 카운터 증가
    selection_counter += 1

    # 모든 POMO가 작업을 완료하고 디팟으로 돌아왔는지 확인
    if done:
        print("\nAll nodes have been visited and returned to the depot. Process completed.")
        break

# 각 드론 경로의 총 이동 거리 계산
total_distances = []
for pomo_idx in range(env_params['pomo_size']):
    drone_path = drone_paths[pomo_idx]
    # 경로에 depot을 추가하여 경로의 시작과 끝이 depot이 되게 함
    drone_path = [depot_xy] + drone_path + [depot_xy]
    
    # 경로 거리 계산
    distance = sum(np.linalg.norm(np.array(drone_path[i]) - np.array(drone_path[i + 1])) for i in range(len(drone_path) - 1))
    total_distances.append(distance)

# 가장 짧은 경로 선택
shortest_path_idx = np.argmin(total_distances)
shortest_path = drone_paths[shortest_path_idx]

# Plotly로 가장 짧은 경로 시각화
fig = go.Figure()

# Depot 표시
fig.add_trace(go.Scatter(x=[depot_xy[0]], y=[depot_xy[1]], mode='markers+text', marker=dict(size=12, color='red'),
                         text=['Depot'], textposition='top center', name='Depot'))

# 모든 노드의 위치와 번호 및 수요 표시
for node_idx in range(1, len(node_positions)):
    demand_text = f"Node {node_idx} (Demand: {node_demands[node_idx - 1]:.2f})"
    fig.add_trace(go.Scatter(x=[node_positions[node_idx][0]], y=[node_positions[node_idx][1]],
                             mode='markers+text', marker=dict(size=8, color='blue'),
                             text=[demand_text], textposition='top center', name=f'Node {node_idx}'))

# 가장 짧은 경로 표시 (depot 추가)
shortest_path = [depot_xy] + shortest_path + [depot_xy]
path_x = [point[0] for point in shortest_path]
path_y = [point[1] for point in shortest_path]

fig.add_trace(go.Scatter(x=path_x, y=path_y, mode='lines+markers', name=f'Shortest Path - Drone {shortest_path_idx + 1}'))

fig.update_layout(title="Shortest Drone Path with Node Numbers and Demand", xaxis_title="X Coordinate", yaxis_title="Y Coordinate")

# 그래프 출력
fig.show()




--- Debugging reset() ---
Depot XY: tensor([[[0.5000, 0.5000]]])
Node XY: tensor([[[0.3708, 0.4093],
         [0.4508, 0.4275],
         [0.3636, 0.4579],
         [0.5859, 0.3165],
         [0.4186, 0.8525],
         [0.3151, 0.3367],
         [0.3270, 0.4195],
         [0.0498, 0.5020],
         [0.8169, 0.1747],
         [0.3797, 0.1441]]])
Node Demand: tensor([[0.6000, 0.0667, 0.6000, 0.3333, 0.0667, 0.6000, 0.4667, 0.6000, 0.2000,
         0.5333]])
Reward: None
Done: False
--------------------------


--- Debugging pre_step() ---
Selected Count: 0
Load: tensor([[1., 1., 1., 1., 1.]])
Battery (SoC): tensor([[95., 95., 95., 95., 95.]])
Current Node: None
Ninf Mask: tensor([[[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]]])
Finished: tensor([[False, False, False, Fals

In [216]:
print("\n--- Debugging reset() ---")
print("Depot XY:", reset_state.depot_xy)
print("Node XY:", reset_state.node_xy)
print("Node Demand:", reset_state.node_demand)
print("Reward:", reward)
print("Done:", done)
print("--------------------------\n")

print("\n--- Debugging pre_step() ---")
print("Selected Count:", step_state.selected_count)
print("Load:", step_state.load)
print("Battery (SoC):", step_state.soc)
print("Current Node:", step_state.current_node)
print("Ninf Mask:", step_state.ninf_mask)
print("Finished:", step_state.finished)
print("Reward:", reward)
print("Done:", done)
print("-----------------------------\n")
step_state, reward, done = env.step(selected)


--- Debugging reset() ---
Depot XY: tensor([[[0.5000, 0.5000]]])
Node XY: tensor([[[0.0110, 0.4176],
         [0.2980, 0.3248],
         [0.9227, 0.9672],
         [0.7573, 0.5258],
         [0.6121, 0.7623],
         [0.1551, 0.3051],
         [0.2630, 0.5092],
         [0.1250, 0.1578],
         [0.6386, 0.5575],
         [0.0040, 0.1307]]])
Node Demand: tensor([[0.6000, 0.0667, 0.5333, 0.5333, 0.1333, 0.5333, 0.0667, 0.4000, 0.5333,
         0.2000]])
Reward: tensor([[-7.0416, -6.2059, -7.0416, -7.1528, -6.6715]])
Done: tensor(True)
--------------------------


--- Debugging pre_step() ---
Selected Count: 19
Load: tensor([[1., 1., 1., 1., 1.]])
Battery (SoC): tensor([[95., 95., 95., 95., 95.]])
Current Node: tensor([[0, 0, 0, 0, 0]])
Ninf Mask: tensor([[[0., -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf],
         [0., -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf],
         [0., -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf],
         [0., -i